In [24]:
MODEL_PATH = "../models/best_model.pkl"
SCALER_PATH = "../models/scaler.pkl"
MODEL_COLUMNS = "../models/model_columns.pkl"

In [25]:
import joblib
import numpy as np
import pandas as pd

In [26]:
loaded_model = joblib.load(MODEL_PATH)
scaler = joblib.load(SCALER_PATH)
model_columns = joblib.load(MODEL_COLUMNS)

In [27]:
print(scaler.feature_names_in_)


['size_sqm' 'bedrooms_num' 'bathrooms_num']


In [39]:
class DATA:
    def __init__(self):
        self.cat_cols = ['type', 'available_from', 'governorate', 'city', 'district', 'compound']
        self.num_cols = ['size_sqm', 'bedrooms_num', 'bathrooms_num']
        self.model = joblib.load(MODEL_PATH)
        self.scaler = joblib.load(SCALER_PATH)
        self.model_columns = joblib.load(MODEL_COLUMNS)

    def preprocess(self, data: dict):
        # Convert dict to DataFrame
        df = pd.DataFrame([data])

        # Drop target column if accidentally included
        if 'price' in df.columns:
            df = df.drop(columns=['price'])

        # Scale numerical features
        df[self.num_cols] = self.scaler.transform(df[self.num_cols])

        # One-hot encode categorical features
        df = pd.get_dummies(df, columns=self.cat_cols)

        # Align columns with model (fill missing columns with 0)
        df = df.reindex(columns=self.model_columns, fill_value=0)

        return df

    def predict_price(self, data: dict):
        df = self.preprocess(data)
        prediction = self.model.predict(df)
        return np.expm1(prediction[0])


# Example usage
if __name__ == "__main__":
    data_input = {
        'type': 'apartment',
        'available_from': '2025-01-10',
        'governorate': 'Alexandria',
        'city': 'Alexandria',
        'district': 'El-Nozha',
        'compound': 'El-Nozha',
        'size_sqm': 300,
        'bedrooms_num': 10,
        'bathrooms_num': 2
    }

    data_handler = DATA()
    predicted_price = data_handler.predict_price(data_input)
    print(f"Predicted Price: {predicted_price} EGP")


Predicted Price: 8043725.980008817 EGP
